In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings("ignore")

In [27]:
df = pd.read_csv('../data/usedCarListing.csv')
del df['Unnamed: 0']
print(df.shape)
df.head()

(9990, 19)


,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,mpg_city,mpg_hwy,engine,transmission,drive_type,fuel_type,popular_feature,vehicle_history,cpo
0,2017,BMW,['X3'],xDrive28i AWD,Marietta,GA,"77,549","['$16,709']",Mineral White Metallic,Mocha,21,28,2.0L Inline-4 Gas Turbocharged,Automatic,AWD,Gas,[],"['0', ' reported accidents', 'Personal or Rent...",False
1,2018,Ford,['F-150'],XLT SuperCrew 5.5' Box 4WD,Deland,FL,"11,624","['$26,985']",Shadow Black,Medium Earth Gray,16,22,2.7L V-6 Gas Turbocharged,Automatic,4WD,Gas,"['Cruise Control', 'Bluetooth', 'Backup Camera']","['0', ' reported accidents', 'Personal or Rent...",False
2,2018,GMC,"['Savana', 'Cargo', 'Van']",2500 Short Wheelbase,Pelham,AL,"23,997","['$19,990']",Summit White,Unknown,N,cty,6.0L V-8 Gas,Automatic,RWD,Gas,"['Cruise Control', 'Backup Camera']","['0', ' reported accidents', 'Personal or Rent...",False
3,2017,Nissan,['Rogue'],SV AWD,Palmetto Bay,FL,"78,116","['$9,997']",Magnetic Black,Charcoal,25,32,2.5L Inline-4 Gas,Automatic,AWD,Gas,"['Front Heated Seats', 'Cruise Control', 'Back...","['2', ' reported accidents', 'Fleet or Mixed U...",False
4,2017,Toyota,['RAV4'],XLE AWD,Great Falls,MT,"189,335","['$13,495']",Super White,Black,22,28,2.5L Inline-4 Gas,Automatic,AWD,Gas,[],"['0', ' reported accidents', 'Personal or Rent...",False


There are totally 9990 used car listings with 19 columns in the datasets. The main columns include:
1. year: model year 
2. make: automaker
3. model: model of the vehicle
4. sub_model: more details of the vehicle model
5. city: current city where the car is located
6. state: current state where the car is located
7. mileage: mileage of the car
8. price: dealers' resell price
9. exterior_color: color of the car
10. interior_color: inside color of the car
11. mpg_city: vehicle efficiency running in city conditions, miles per gallon
12. mpg_hwy: vehicle efficiency running in highway conditions, miles per gallon
13. engine: engine specification
14. transmission: automatic or manual transmission type
15. drive_type: drive type of the car
16. fuel_type: fuel type of the car, gasolin/diesel/hybrid/electric etc.
17. popular_feature: the popular features that the car contains
18. vehicle_history: a vector of vehicle past history including number of reported accidents, number of previous owners, title, and usage type.
19. cpo: whether the car is certified pre-owned or not.

## 1. Create new features

In [28]:
# split vehicle history into four variables(num_accident, usage_type, title, num_owner)
split_data = df["vehicle_history"].str.split(",")
data = split_data.to_list()
names = ["1", "2", "3", "4", '5', '6']
new_df = pd.DataFrame(data, columns=names)
df['num_accident'] = new_df['1'].apply(lambda x:x[2:-1])
df['usage_type'] = new_df['3'].apply(lambda x:x[2:-1]) 
df['title'] = new_df['4'].apply(lambda x:x[2:-1])
df['num_owner'] = new_df['5'].apply(lambda x:x[2:-2])
# delete the history column
del df['vehicle_history']
df.head()

,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,...,engine,transmission,drive_type,fuel_type,popular_feature,cpo,num_accident,usage_type,title,num_owner
0,2017,BMW,['X3'],xDrive28i AWD,Marietta,GA,"77,549","['$16,709']",Mineral White Metallic,Mocha,...,2.0L Inline-4 Gas Turbocharged,Automatic,AWD,Gas,[],False,0,Personal or Rental Use,Clean,1
1,2018,Ford,['F-150'],XLT SuperCrew 5.5' Box 4WD,Deland,FL,"11,624","['$26,985']",Shadow Black,Medium Earth Gray,...,2.7L V-6 Gas Turbocharged,Automatic,4WD,Gas,"['Cruise Control', 'Bluetooth', 'Backup Camera']",False,0,Personal or Rental Use,Clean,1
2,2018,GMC,"['Savana', 'Cargo', 'Van']",2500 Short Wheelbase,Pelham,AL,"23,997","['$19,990']",Summit White,Unknown,...,6.0L V-8 Gas,Automatic,RWD,Gas,"['Cruise Control', 'Backup Camera']",False,0,Personal or Rental Use,Clean,1
3,2017,Nissan,['Rogue'],SV AWD,Palmetto Bay,FL,"78,116","['$9,997']",Magnetic Black,Charcoal,...,2.5L Inline-4 Gas,Automatic,AWD,Gas,"['Front Heated Seats', 'Cruise Control', 'Back...",False,2,Fleet or Mixed Use,Clean,1
4,2017,Toyota,['RAV4'],XLE AWD,Great Falls,MT,"189,335","['$13,495']",Super White,Black,...,2.5L Inline-4 Gas,Automatic,AWD,Gas,[],False,0,Personal or Rental Use,Clean,1


By splitting the vehicle_history columns into four features, which can greatly represent the vehicle historical information.
1. num_accident: number of reported accidents of the car
2. usage_type: whether the car is mainly used for fleet/mixed use or personal/rental use.
3. title: whether this car has a clean, salvage or other kinds of titles.
4. num_owner: number of previous owners of the used car.

## Check the unique values of four newly constructed features

In [29]:
df.num_owner.unique(),df.num_accident.unique(),df.title.unique(),df.usage_type.unique()

(array(['1', '2', '5', '3', '4', '0', '6', '8', '7', '9', '13', 'alvage',
        'rame Damage'], dtype=object),
 array(['0', '2', '1', '3', '4', '5'], dtype=object),
 array(['Clean', 'Salvaged', 'Frame Damaged', 'Theft Recovered',
        'Theft Recovere', 'Lemon', 'Frame Damage'], dtype=object),
 array(['Personal or Rental Use', 'Fleet or Mixed Use'], dtype=object))

I found that there are some wrong values or synonyms('Frame Damaged', 'Frame Damage') in num_owner, title features due to scraping or parsing. The next block is to deal with these wrong data.

In [30]:
### Deal with title (combine theft recovered with theft recovere, combine frame damaged with frame damage)
df["title"]= df["title"].replace('Theft Recovere', "Theft Recovered") 
df["title"]= df["title"].replace('Frame Damage', "Frame Damaged") 
print(df.title.unique())

### Deal with num_owner that are strings
df["num_owner"]= df["num_owner"].replace('alvage', np.nan) 
df["num_owner"]= df["num_owner"].replace('rame Damage', np.nan) 
print(df.num_owner.unique())

['Clean' 'Salvaged' 'Frame Damaged' 'Theft Recovered' 'Lemon']
['1' '2' '5' '3' '4' '0' '6' '8' '7' '9' '13' nan]


## 2. Check for missing values

In [31]:
df.isna().sum()

year               0
make               0
model              0
sub_model          0
city               0
state              0
mileage            0
price              0
exterior_color     0
interior_color     0
mpg_city           0
mpg_hwy            0
engine             0
transmission       0
drive_type         0
fuel_type          0
popular_feature    0
cpo                0
num_accident       0
usage_type         0
title              0
num_owner          4
dtype: int64

In [32]:
# num_owern, numeric missing value, take the median to fillna
df['num_owner'].fillna(df['num_owner'].median(), inplace=True)

In [33]:
## look at mpg features
print(df.mpg_city.unique())
print(df.mpg_hwy.unique())

['21' '16' 'N' '25' '22' '24' '31' '17' '18' '49' '26' '28' '23' '15' '12'
 '19' '20' '126' '27' '29' '14' '30' '13' '44' '101' '43' '66' '104' '121'
 '120' '32' '48' '38' '95' '51' '106' '41' '40' '35' '11' '110' '129' '42'
 '85' '10' '34' '122' '33' '53' '36' '39' '128' '60' '91' '54' '124' '37'
 '8' '137']
['28' '22' 'cty' '32' '30' '33' '40' '23' '25' '47' '38' '31' '36' '34'
 '29' '24' '21' '16' '99' '37' '39' '19' '17' '26' '35' '18' '27' '20'
 '44' '93' '41' '66' '91' '103' '92' '42' '45' '15' '81' '48' '43' '102'
 '105' '82' '49' '46' '109' '51' '53' '100' '90' '50' '94' '101' '14' '13'
 '12' '111' '108']


Found that there are 'N' and 'cty' values in the mpg features, which are wrong data that we need to replace with NA.

In [34]:
## convert 'N' and 'cty' to nan.
df["mpg_city"]= df["mpg_city"].replace('N', np.nan) 
df["mpg_hwy"]= df["mpg_hwy"].replace('cty', np.nan) 

In [35]:
# deal with mpg_city, mpg_hwy
# first convert mpg from string to numeric values
df['mpg_city'] = df['mpg_city'].astype(float)
df['mpg_hwy'] = df['mpg_hwy'].astype(float)

## Fill missing values in mpg based on domain knowledge.
Since mpg is strongly correlated with vehicle fuel_type, especially for alternative fuel vehicles (hybrid, electric, CNG, hydrogen), I will use the group mean to fillna for these types of vehicles.

For gasoline and diesel vehicles, by observation, I found the missing mpg values for these types of vehicles are more like gas-gazzlers. Therefore, I decided to use 5% quantiles to fill these missing values to indicate their extreme low mpgs.

In [40]:
# checking the fuel types that contains missing values
df[df['mpg_city'].isna()]['fuel_type'].unique()

array(['Gas', 'Hydrogen', 'Diesel', 'Plug-In', 'CNG', 'Hybrid'],
      dtype=object)

In [41]:
# checking gasoline and diesel vehicles whose mpg are missing
display(df[(df['fuel_type']=='Gas') & (df['mpg_city'].isna())].head())
display(df[(df['fuel_type']=='Diesel') & (df['mpg_city'].isna())].head())

,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,...,engine,transmission,drive_type,fuel_type,popular_feature,cpo,num_accident,usage_type,title,num_owner
2,2018,GMC,"['Savana', 'Cargo', 'Van']",2500 Short Wheelbase,Pelham,AL,"23,997","['$19,990']",Summit White,Unknown,...,6.0L V-8 Gas,Automatic,RWD,Gas,"['Cruise Control', 'Backup Camera']",False,0,Personal or Rental Use,Clean,1
34,2017,Ford,"['Super', 'Duty', 'F-250']",XL Crew Cab 8' Bed 4WD,Lexington,NC,"94,903","['$29,987']",White,Medium Earth Gray,...,6.2L V-8 Gas,Automatic,4WD,Gas,[],False,0,Personal or Rental Use,Clean,1
180,2004,HUMMER,['H2'],SUV,McCook,IL,"149,795","['$9,950']",White,Wheat,...,6.0L V-8 Gas,Automatic,4WD,Gas,[],False,2,Personal or Rental Use,Clean,1
185,2017,Ford,"['Super', 'Duty', 'F-250']",XLT Crew Cab 6.75' Bed 4WD,Huntersville,NC,"62,669","['$34,787']",Shadow Black,Medium Earth Gray,...,6.2L V-8 Gas,Automatic,4WD,Gas,[],False,0,Personal or Rental Use,Clean,1
218,2004,Dodge,"['Ram', '2500']",SLT Quad Cab Regular Bed 4WD,Colorado Springs,CO,"159,551","['$10,300']",Gray,Black,...,5.7L V-8 Gas,Automatic,4WD,Gas,[],False,0,Fleet or Mixed Use,Clean,3


,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,...,engine,transmission,drive_type,fuel_type,popular_feature,cpo,num_accident,usage_type,title,num_owner
203,2017,Ford,"['Super', 'Duty', 'F-250']",Lariat Crew Cab 8' Bed 4WD,Columbus,OH,"18,095","['$34,999']",White,Unspecified,...,6.7L V-8 Diesel Turbocharged,Automatic,4WD,Diesel,[],False,1,Fleet or Mixed Use,Frame Damaged,2
269,2019,Mercedes-Benz,"['Sprinter', 'Cab', 'Chassis']","3500XD 170""",Houston,TX,10,"['$33,995']",White,Gray,...,3.0L V-6 Diesel Turbocharged,Automatic,RWD,Diesel,[],False,0,Personal or Rental Use,Clean,1
270,2013,Mercedes-Benz,"['Sprinter', 'Passenger', 'Van']",2500 High Roof LWB,National City,CA,"159,212","['$18,995']",Gray,Black,...,3.0L V-6 Diesel Turbocharged,Automatic,RWD,Diesel,[],False,0,Fleet or Mixed Use,Clean,2
292,2017,Ford,"['Super', 'Duty', 'F-350']",Platinum Crew Cab 8' Bed 4WD SRW,Hillsboro,OR,"38,755","['$42,995']",White,Unspecified,...,6.7L V-8 Diesel Turbocharged,Automatic,4WD,Diesel,[],False,0,Personal or Rental Use,Clean,1
322,2019,Mercedes-Benz,"['Sprinter', 'Cab', 'Chassis']","3500XD 170""",Machesney Park,IL,300,"['$32,979']",Grey White,Black,...,3.0L V-6 Diesel Turbocharged,Automatic,RWD,Diesel,[],False,0,Personal or Rental Use,Clean,1


Diesel and gasoline cars whose mpgs are missing are either large cars or luxury cars, whose mpg will be at the very low level.

## Fillna for mpg features

In [42]:
# use fuel_type (electric, plug-in, hybrid, hydrogen, CNG) to fillna first. 
# deal with mpg_city first.
for i in range(len(df)):
    if (df['fuel_type'][i] == 'Hydrogen' and np.isnan(df['mpg_city'][i])):
        df['mpg_city'][i]= df[df['fuel_type']=='Hydrogen']['mpg_city'].mean()
    elif (df['fuel_type'][i] == 'Plug-In' and np.isnan(df['mpg_city'][i])):
        df['mpg_city'][i]=df[df['fuel_type']=='Plug-In']['mpg_city'].mean()
    elif (df['fuel_type'][i] == 'Hybrid' and np.isnan(df['mpg_city'][i])):
        df['mpg_city'][i]=df[df['fuel_type']=='Hybrid']['mpg_city'].mean()
    elif (df['fuel_type'][i] == 'CNG' and np.isnan(df['mpg_city'][i])):
        df['mpg_city'][i]=df[df['fuel_type']=='CNG']['mpg_city'].mean()
    # The missing values for Diesel and Gas vehicles are more like gas-gazzlers. So I can't use mean or median
    # to replace the missing values. 
    # I decided to use the 95% quantile values to fillna
    elif (df['fuel_type'][i] == 'Gas' and np.isnan(df['mpg_city'][i])):
        df['mpg_city'][i]=df[df['fuel_type']=='Gas']['mpg_city'].quantile(q=0.05)
    elif (df['fuel_type'][i] == 'Diesel' and np.isnan(df['mpg_city'][i])):
        df['mpg_city'][i]=df[df['fuel_type']=='Diesel']['mpg_city'].quantile(q=0.05)
    
# then deal with mpg_hwy.      
for i in range(len(df)):
    if (df['fuel_type'][i] == 'Hydrogen' and np.isnan(df['mpg_hwy'][i])):
        df['mpg_hwy'][i]=df[df['fuel_type']=='Hydrogen']['mpg_hwy'].mean()
    elif (df['fuel_type'][i] == 'Plug-In' and np.isnan(df['mpg_hwy'][i])):
        df['mpg_hwy'][i]=df[df['fuel_type']=='Plug-In']['mpg_hwy'].mean()
    elif (df['fuel_type'][i] == 'Hybrid' and np.isnan(df['mpg_hwy'][i])):
        df['mpg_hwy'][i]=df[df['fuel_type']=='Hybrid']['mpg_hwy'].mean()        
    elif (df['fuel_type'][i] == 'CNG' and np.isnan(df['mpg_hwy'][i])):
        df['mpg_hwy'][i]=df[df['fuel_type']=='CNG']['mpg_hwy'].mean()
    # For diesel vehicles with missing values, I found all diesel vehicles are missing the mpg_hwy. 
    elif (df['fuel_type'][i] == 'Gas' and np.isnan(df['mpg_hwy'][i])):
        df['mpg_hwy'][i]=df[df['fuel_type']=='Gas']['mpg_hwy'].quantile(q=0.05)
    elif (df['fuel_type'][i] == 'Diesel' and np.isnan(df['mpg_hwy'][i])):
        df['mpg_hwy'][i]=df[df['fuel_type']=='Diesel']['mpg_hwy'].quantile(q=0.05)       
        
df.head()

,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,...,engine,transmission,drive_type,fuel_type,popular_feature,cpo,num_accident,usage_type,title,num_owner
0,2017,BMW,['X3'],xDrive28i AWD,Marietta,GA,"77,549","['$16,709']",Mineral White Metallic,Mocha,...,2.0L Inline-4 Gas Turbocharged,Automatic,AWD,Gas,[],False,0,Personal or Rental Use,Clean,1
1,2018,Ford,['F-150'],XLT SuperCrew 5.5' Box 4WD,Deland,FL,"11,624","['$26,985']",Shadow Black,Medium Earth Gray,...,2.7L V-6 Gas Turbocharged,Automatic,4WD,Gas,"['Cruise Control', 'Bluetooth', 'Backup Camera']",False,0,Personal or Rental Use,Clean,1
2,2018,GMC,"['Savana', 'Cargo', 'Van']",2500 Short Wheelbase,Pelham,AL,"23,997","['$19,990']",Summit White,Unknown,...,6.0L V-8 Gas,Automatic,RWD,Gas,"['Cruise Control', 'Backup Camera']",False,0,Personal or Rental Use,Clean,1
3,2017,Nissan,['Rogue'],SV AWD,Palmetto Bay,FL,"78,116","['$9,997']",Magnetic Black,Charcoal,...,2.5L Inline-4 Gas,Automatic,AWD,Gas,"['Front Heated Seats', 'Cruise Control', 'Back...",False,2,Fleet or Mixed Use,Clean,1
4,2017,Toyota,['RAV4'],XLE AWD,Great Falls,MT,"189,335","['$13,495']",Super White,Black,...,2.5L Inline-4 Gas,Automatic,AWD,Gas,[],False,0,Personal or Rental Use,Clean,1


In [43]:
## no more NA for now
df.isna().sum()

year               0
make               0
model              0
sub_model          0
city               0
state              0
mileage            0
price              0
exterior_color     0
interior_color     0
mpg_city           0
mpg_hwy            0
engine             0
transmission       0
drive_type         0
fuel_type          0
popular_feature    0
cpo                0
num_accident       0
usage_type         0
title              0
num_owner          0
dtype: int64

## 3. Convert some categorical variables into numeric
Numeric variable: mileage, price, mpg_city, mpg_hwy, num_accident, num_owner

## deal with mileage for the comma.

In [45]:
df['mileage'] = df['mileage'].apply(lambda x: ''.join(x.split(',')))
df['mileage'] = df['mileage'].astype(int)
df['mileage'].head()

0     77549
1     11624
2     23997
3     78116
4    189335
Name: mileage, dtype: int64

## deal with price

In [46]:
# deal with price
df['price'] = df['price'].apply(lambda x: ''.join(x[3:-2].split(',')))
for i in range(len(df)):
    if len(df['price'][i])==0:
        empty_index = i 
    else:
        df['price'][i]= int(df['price'][i])
df['price'].head()

0    16709
1    26985
2    19990
3     9997
4    13495
Name: price, dtype: object

In [47]:
# drop the rows with missing price information
print(empty_index)
df = df.drop(df.index[empty_index])
print(df.shape)

2332
(9989, 22)


## deal with num_accident and num_owner

In [49]:
df['num_accident'] = df['num_accident'].astype(int)
df['num_owner'] = df['num_owner'].astype(int)

In [50]:
# checking feature types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9989 entries, 0 to 9989
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             9989 non-null   int64  
 1   make             9989 non-null   object 
 2   model            9989 non-null   object 
 3   sub_model        9989 non-null   object 
 4   city             9989 non-null   object 
 5   state            9989 non-null   object 
 6   mileage          9989 non-null   int64  
 7   price            9989 non-null   object 
 8   exterior_color   9989 non-null   object 
 9   interior_color   9989 non-null   object 
 10  mpg_city         9989 non-null   float64
 11  mpg_hwy          9989 non-null   float64
 12  engine           9989 non-null   object 
 13  transmission     9989 non-null   object 
 14  drive_type       9989 non-null   object 
 15  fuel_type        9989 non-null   object 
 16  popular_feature  9989 non-null   object 
 17  cpo           

## 4. Drop duplicate rows

In [51]:
# There are totally 5 duplcated rows
print(df.duplicated().sum())
# drop duplicated rows and keep the first row.
print(df.shape)
df = df.drop_duplicates(keep='first')
print(df.shape)

5
(9989, 22)
(9984, 22)


## 5. Deal with potential outliers/wrong data (mileage, price)

In [21]:
# Check very low mileage and very high mileage
display(df.sort_values(by='mileage',ascending=False).head())
df.sort_values(by='mileage',ascending=True).head()
## The data seems making sense to me. I won't drop any data from mileage here.

,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,...,engine,transmission,drive_type,fuel_type,popular_feature,cpo,num_accident,usage_type,title,num_owner
3091,2012,Ram,['3500'],SLT Crew Cab 8' Box 2WD,Charlotte,NC,484773,14995,Bright White,Dark Slate/Medium Graystone Interior,...,6.7L Inline-6 Diesel Turbocharged,Automatic,RWD,Diesel,"['Remote Engine Start', 'Cruise Control']",False,1,Fleet or Mixed Use,Clean,2
6988,2012,Volkswagen,['Jetta'],TDI SportWagen DSG,Kingsport,TN,445612,4900,Silver,Titan Black,...,2.0L Inline-4 Diesel Turbocharged,Automatic,FWD,Diesel,[],False,1,Personal or Rental Use,Clean,1
7744,2006,Toyota,['Tundra'],SR5 Double Cab V8 RWD Automatic,San Antonio,TX,381822,4499,Salsa Red Pearl,Taupe,...,4.7L V-8 Gas,Automatic,RWD,Gas,[],False,0,Personal or Rental Use,Clean,2
8430,2001,Chevrolet,"['Silverado', '1500']",LT Extended Cab Standard Box 2WD,Pensacola,FL,366778,3985,Silver,Unknown,...,5.3L V-8 Gas,Automatic,RWD,Gas,[],False,0,Personal or Rental Use,Clean,2
1725,2012,Ford,"['Super', 'Duty', 'F-450']","Lariat Crew Cab 172"" DRW 4WD",Phoenix,AZ,366633,15888,White,Unknown,...,6.7L V-8 Diesel Turbocharged,Automatic,4WD,Diesel,[],False,0,Fleet or Mixed Use,Clean,1


,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,...,engine,transmission,drive_type,fuel_type,popular_feature,cpo,num_accident,usage_type,title,num_owner
4438,2019,Dodge,"['Grand', 'Caravan']",SXT,Roanoke,IL,6,21923,White Knuckle Clearcoat,Black/Light Graystone,...,3.6L V-6 Gas,Automatic,FWD,Gas,"['Power Trunk/Liftgate', 'Remote Engine Start'...",False,0,Personal or Rental Use,Clean,0
8562,2019,Mercedes-Benz,['A-Class'],A 220 4MATIC,Lincolnwood,IL,7,33875,Night Black,Black,...,2.0L Inline-4 Gas Turbocharged,Automatic,AWD,Gas,[],True,0,Personal or Rental Use,Clean,0
2043,2019,Mercedes-Benz,"['Sprinter', 'Cab', 'Chassis']","4500 170""",Monroe,NC,8,28998,White,Unknown,...,3.0L V-6 Diesel Turbocharged,Automatic,RWD,Diesel,[],False,0,Personal or Rental Use,Clean,0
5304,2020,Mercedes-Benz,['C-Class'],C 300 Sedan 4MATIC,Lake Bluff,IL,9,39990,Black,Silk Beige,...,2.0L Inline-4 Gas Turbocharged,Automatic,AWD,Gas,[],False,0,Personal or Rental Use,Clean,0
1996,2019,Mercedes-Benz,"['Sprinter', 'Cab', 'Chassis']","4500 170""",Monroe,NC,10,28998,White,Unknown,...,3.0L V-6 Diesel Turbocharged,Automatic,RWD,Diesel,[],False,0,Personal or Rental Use,Clean,0


By checking the 10 cars with top 5 highest mileage and 5 lowest mileage, I need to check if they are making sense.

For top 5 mileage, the total mileage for 8 years is about 484k and average annua mileage is about 60k, which totally makes sense to me.

For bottom 5 mileage, the mileage is from 6~10, which is a very low number. By checking the model year, num_owner and usage_type, I guess these vehicles are purchased by car rental companies and haven't been driven ever, but due to some known reasons, these cars are listed as used cars online for resell purpose. These data makes sense to me.

Therefore, I keep all the data records at this step.

In [22]:
# Check very low price and very high price
display(df.sort_values(by='price',ascending=False).head(10))
df.sort_values(by='price',ascending=True).head(10)
## The high and low prices seems making sense to me. Luxury used cars have very high price. 
## Old used cars have very low prices
## I won't drop any data from mileage here.

,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,...,engine,transmission,drive_type,fuel_type,popular_feature,cpo,num_accident,usage_type,title,num_owner
7204,2015,Rolls-Royce,['Wraith'],RWD,Tempe,AZ,23390,149888,Black,Black,...,6.6L V-12 Gas Turbocharged,Automatic,RWD,Gas,[],False,0,Personal or Rental Use,Clean,1
7736,2005,Lamborghini,['Murcielago'],Coupe,Boerne,TX,4565,148888,Grey Metallic,Red,...,6.2L V-12 Gas,Automatic,AWD,Gas,[],False,0,Personal or Rental Use,Clean,2
5159,2014,Ferrari,['FF'],Hatchback,Greensboro,NC,15237,144635,Nero,Nero,...,6.3L V-12 Gas,Automatic,AWD,Gas,[],True,0,Fleet or Mixed Use,Clean,3
1690,2010,Ferrari,"['599', 'GTB', 'Fiorano']",Coupe,Ontario,CA,30061,139500,Nero Daytona Metallic,Nero,...,6.0L V-12 Gas,Manual,RWD,Gas,[],False,0,Personal or Rental Use,Clean,5
650,2008,Ferrari,"['599', 'GTB', 'Fiorano']",Coupe,Boerne,TX,18075,129888,Corsa Red,Interior Color,...,6.0L V-12 Gas,Automatic,RWD,Gas,[],False,0,Personal or Rental Use,Clean,5
696,2015,Ferrari,['California'],Convertible,Melbourne,FL,5835,128998,Nero Daytona Metallic,Sabbia,...,3.9L V-8 Gas Turbocharged,Automatic,RWD,Gas,[],False,0,Personal or Rental Use,Clean,2
7698,2008,Ferrari,"['599', 'GTB', 'Fiorano']",Coupe,Marietta,GA,16148,124900,Silverstone Gray,Interior Color,...,6.0L V-12 Gas,Manual,RWD,Gas,[],False,0,Personal or Rental Use,Clean,4
2501,2012,Ferrari,['FF'],Hatchback,Pittsburgh,PA,25400,118075,Gray,Tan,...,6.3L V-12 Gas,Automatic,AWD,Gas,[],False,0,Personal or Rental Use,Clean,4
5563,2007,Ferrari,"['599', 'GTB', 'Fiorano']",Coupe,Mesa,AZ,18687,114999,Daytona Black,Black,...,6.0L V-12 Gas,Automatic,RWD,Gas,[],False,0,Personal or Rental Use,Clean,5
4036,2008,Bentley,['Azure'],Convertible,Naples,FL,10393,109995,Beluga Black,Standard Interior,...,6.8L V-8 Gas Turbocharged,Automatic,RWD,Gas,[],False,0,Personal or Rental Use,Clean,2


,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,...,engine,transmission,drive_type,fuel_type,popular_feature,cpo,num_accident,usage_type,title,num_owner
253,2005,Dodge,['Caravan'],Grand SXT LWB,Chicago,IL,183245,1500,Stone White,Unknown,...,3.8L V-6 Gas,Automatic,FWD,Gas,['Cruise Control'],False,1,Fleet or Mixed Use,Clean,4
710,1999,Suzuki,"['Grand', 'Vitara']",4dr JLX Auto 4WD,Manchester,MD,180410,1500,Black,Gray,...,2.5L V-6 Gas,Automatic,4WD,Gas,[],False,0,Personal or Rental Use,Clean,4
8790,2001,Ford,['Escape'],XLT FWD,Plainfield,IN,254999,1538,Toreador Red Metallic,Medium Graphite,...,3.0L V-6 Gas,Automatic,FWD,Gas,['Cruise Control'],False,3,Personal or Rental Use,Clean,8
6055,2000,Ford,['Escort'],ZX2 Coupe,Everett,WA,287225,1590,Silver Frost Metallic,Dark Charcoal,...,2.0L Inline-4 Gas,Manual,FWD,Gas,[],False,0,Personal or Rental Use,Clean,3
6784,2002,Honda,['Civic'],LX Coupe Automatic,Orlando,FL,245421,1595,Taffeta White,Gray,...,1.7L Inline-4 Gas,Automatic,FWD,Gas,['Cruise Control'],False,0,Personal or Rental Use,Clean,4
3623,1998,Chevrolet,"['Monte', 'Carlo']",LS,Kent,WA,188000,1600,Torch Red,Black,...,3.1L V-6 Gas,Automatic,FWD,Gas,[],False,0,Personal or Rental Use,Clean,6
5857,2001,Honda,['Odyssey'],EX 7-Passenger,Bloomfield Hills,MI,233255,1695,Dark Emerald Pearl,Ivory,...,3.5L V-6 Gas,Automatic,FWD,Gas,['Cruise Control'],False,1,Personal or Rental Use,Clean,1
2667,1998,Dodge,['Caravan'],Base 3-door FWD SWB,Boaz,AL,215798,1800,Blue,Unknown,...,3.0L V-6 Gas,Automatic,FWD,Gas,[],False,0,Personal or Rental Use,Clean,3
1166,2003,Buick,['Regal'],LS,Lakewood,NJ,128198,1800,Sterling Silver Metallic,Medium Gray,...,3.8L V-6 Gas,Automatic,FWD,Gas,['Cruise Control'],False,0,Personal or Rental Use,Clean,4
5777,2002,Ford,['Mustang'],Standard Coupe,Happy Valley,OR,233850,1850,Black,Black,...,3.8L V-6 Gas,Manual,RWD,Gas,[],False,0,Personal or Rental Use,Clean,2


By checking the top 10 and bottom 10 used cars in term of resell price, I found the 10 most expensive cars are all luxury brands such as Rolls-Royce, Ferrari, Bentley and Lamborghini, which makes sense for the extreme high prices.

For the 10 cheapest used cars, they are old cars with very high mileage. Therefore, the low prices make sense here, too.

Therefore, I will also keep all the data from this step.

In [52]:
print(df.shape)

(9984, 22)


In [53]:
df.to_csv('../data/usedCarListingCleaned.csv', encoding = 'utf-8')

After the feature engineering(creating new features and dropping features), dealing with missing values, checking outliers, dropping duplicated rows, there are totally 9984 records and 22 features left.